# Implémentation de la méthode Crank-Nicolson

# Introduction

Dans cette section, nous allons implémenter la méthode de Crank-Nicolson avec les données mises à dispostion. Par conséquent, un traitement de ces données est requis en premier lieu. Celui-ci se fera par cycle : journée, mois, année. Il sera aussi nécessaire de déterminer les températures max et min sur chacun des cycles pour le calcul des conditions limites. 

Puis après, nous mettrons en place la résolution numérique en utilisant la méthode implicite de Crank-Nicolson.

NB : Pour simplifier le problème, la structure des cycles sera comme ceci : 1 année = 12 mois, 1 mois = 30 jours, 1 jour = 24 heures.

Tout d'abord avant de commencer à travailler, importons les libraires et les variables dont on aura besoin.

In [24]:
# importation des librairies 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import Axes3D

On récupère les données dans les fichiers sources et on les sauvegarde dans des listes pour pouvoir les traiter : 

In [25]:
#Algérie 

list_Alg = []

with open("Donnees/temperatures_algeria_1.txt","r") as algerie_1:
    line = algerie_1.readline()
    while line != "":
        data = float(line)
        list_Alg.append(data)
        line = algerie_1.readline()

#France

list_Fr = []

with open("Donnees/temperatures_france_1.txt","r") as france_1:
    
    line = france_1.readline()
    while line != "":
        data = float(line)
        list_Fr.append(data)
        line = france_1.readline()

Puis on déclare les dictionnaires de stockages pour chacun des cycles : 

In [26]:
#Cycle sur une année
list_years = {"Algérie":[], "France":[]}
#Cycle sur un mois
list_months = {"Algérie":[], "France":[]}
#Cycle sur une journée
list_days = {"Algérie":[], "France":[]}

# Cycle sur une année : 

Ici, nous allons faire la moyenne des températures sur une année, i.e sommer toutes les températures puis diviser par le nombre d'éléments sommés.

In [27]:
list_years['Algérie'] = np.mean(list_Alg)
list_years['France'] = np.mean(list_Fr)

# Cycle sur un mois : 

Ici, nous allons faire la moyenne des températures sur une année, i.e sommer toutes les températures d'un mois puis diviser par le nombre d'éléments sommés.

In [28]:
#Algérie

sum_month = 0.0 
hour = 1
    
for data in list_Alg :
    sum_month += data 
    if hour == 730:
        average_temp = sum_month/(24*30)
        list_months['Algérie'].append(average_temp)
        average_temp = 0
        hour = 0
        sum_month = 0
    hour += 1
    
#France

sum_month = 0.0 
hour = 1
    
for data in list_Fr :
    sum_month += data 
    if hour == 730:
        average_temp = sum_month/(24*30)
        list_months['France'].append(average_temp)
        average_temp = 0
        hour = 0
        sum_month = 0
    hour += 1

# Cycle sur une journée :

Ici, nous allons faire la moyenne des températures sur une journée, i.e sommer toutes les températures d'une journée puis diviser par le nombre d'éléments sommés.

In [29]:
#Algérie

sum_day = 0.0 
hour = 1
    
for data in list_Alg:
    sum_day += data 
    if hour == 24:
        average_temp = sum_day/24
        list_days['Algérie'].append(average_temp)
        average_temp = 0
        hour = 0
        sum_day = 0
    hour += 1


#France 

sum_day = 0.0 
hour = 1
    
for data in list_Fr:
    sum_day += data 
    if hour == 24:
        average_temp = sum_day/24
        list_days['France'].append(average_temp)
        average_temp = 0
        hour = 0
        sum_day = 0
    hour += 1

# Détermination de Tmin et Tmax pour chaque cycle

Dans cette partie, nous allons déterminer les températures max et min pour chaque cycle afin de pouvoir calculer l'une des conditions aux limites, T0. Pour cela, on va créer plusieurs dictionnaires et listes : 

- une liste T0_years qui contiendra deux éléments : les températures max et min d'une année.
- un dictionnaire T0_months qui pour chaque mois contiendra les températures max et min de celui-ci. 
- un dictionnaire T0_days qui pour chaque jour contiendra les températures max et min de celui-ci.

In [65]:
# Cycle année

T0_years = {"Algérie":[], "France":[]}

T0_years['Algérie'].append(np.max(list_months['Algérie']))
T0_years['Algérie'].append(np.min(list_months['Algérie']))
T0_years['France'].append(np.max(list_months['France']))
T0_years['France'].append(np.min(list_months['France']))

# Cycle mois 

T0_months = {"Algérie":{
    "janvier": [],
    "février": [],
    "mars": [],
    "avril": [],
    "mai": [],
    "juin": [],
    "juillet": [],
    "août": [],
    "septembre": [],
    "octobre": [],
    "novembre": [],
    "décembre": []
}, "France":{
    "janvier": [],
    "février": [],
    "mars": [],
    "avril": [],
    "mai": [],
    "juin": [],
    "juillet": [],
    "août": [],
    "septembre": [],
    "octobre": [],
    "novembre": [],
    "décembre": []
}}

dict_months = {1 : "janvier",
    2 : "février",
    3 : "mars",
    4 : "avril",
    5 : "mai",
    6 : "juin",
    7 : "juillet",
    8 : "août",
    9 : "septembre",
    10 : "octobre",
    11 : "novembre",
    12 : "décembre"}

temp_list = []
i = 1

for data in list_days['Algérie']:
    if i == 13:
        break
    if len(temp_list) == 30:
        T0_months['Algérie'][dict_months[i]].append(np.max(temp_list))
        T0_months['Algérie'][dict_months[i]].append(np.min(temp_list))
        temp_list.clear()
        i += 1
    temp_list.append(data)
    
temp_list.clear()
i = 1

for data in list_days['France']:
    if i == 13:
        break
    if len(temp_list) == 30:
        T0_months['France'][dict_months[i]].append(np.max(temp_list))
        T0_months['France'][dict_months[i]].append(np.min(temp_list))
        temp_list.clear()
        i += 1
    temp_list.append(data)
    
# Cycle journée 

T0_days = {"Algérie":{
"janvier": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"février": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"mars": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"avril": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"mai": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"juin": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"juillet": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"août": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"septembre": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"octobre": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"novembre": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"décembre": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
}},"France":{
"janvier": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"février": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"mars": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"avril": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"mai": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"juin": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"juillet": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"août": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"septembre": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"octobre": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"novembre": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
},"décembre": {1 : [],
2 : [],
3 : [],
4 : [],
5 : [],
6 : [],
7 : [],
8 : [],
9 : [],
10 : [],
11 : [],
12 : [],
13 : [],
14 : [],
15 : [],
16 : [],
17 : [],
18 : [],
19 : [],
20 : [],
21 : [],
22 : [],
23 : [],
24 : [],
25 : [],
26 : [],
27 : [],
28 : [],
29 : [],
30 : [],
}}}

temp_list.clear()
i = 1
j = 1

for data in list_Alg:
    if j == 13:
        break
    if i == 30:
        j += 1
    if len(temp_list) == 24:
        T0_days['Algérie'][dict_months[j]][i].append(np.max(temp_list))
        T0_days['Algérie'][dict_months[j]][i].append(np.min(temp_list))
        temp_list.clear()
        i += 1
    temp_list.append(data)

temp_list.clear()
i = 1
j = 1

for data in list_Fr:
    if j == 13:
        break
    if i == 30:
        j += 1
    if len(temp_list) == 24:
        T0_days['France'][dict_months[j]][i].append(np.max(temp_list))
        T0_days['France'][dict_months[j]][i].append(np.min(temp_list))
        temp_list.clear()
        i += 1
    temp_list.append(data)

# Implémentation du schéma de Crank-Nicolson 

Dans cette partie, nous allons enfin mettre en place la résolution numérique de l'équation de chaleur suivant le schéma implicite de Crank-Nicolson. 

Nous devons dans un premier temps, déclarer les constantes physiques, les paramètres de la simulation et une fonction tridiag permettant de créer une matrice triadiagonale : 

In [86]:
#fonction triadiagonale 
def triadiag(n,c1,c2):
    M = np.zeros((n+1,n+1),float)
    M[n,n] = c2
    M[n,n-1] = c1
    for i in range (n):
        M[i,i]= c2
        M[i,i-1] = M[i,i+1]= c1
    
    return M

#Constantes physiques et paramétres de la simulation : 

#Algérie (sable sec), source : https://fr.wikipedia.org/wiki/Diffusivit%C3%A9_thermique
rho_Alg = 1650 #masse volumique
c_Alg = 800 #capacité thermique
lamb_Alg = 0.27 #conductivité thermique

D_Alg = lamb_Alg/(rho_Alg*c_Alg) #diffusivité thermique 

#France (argile), source : https://fr.wikipedia.org/wiki/Diffusivit%C3%A9_thermique
rho_Fr = 1450 #masse volumique
c_Fr = 880 #capacité thermique
lamb_Fr = 1.28 #conductivité thermique

D_Fr = lamb_Fr/(rho_Fr*c_Fr) #diffusivité thermique 

#période (une journée)
t = 24*60*60
omega = (2*np.pi)/t
a = lamb_Alg/(omega*c_Alg)

#paramètres de l'expérience
t_fin = 
L = 3*sqrt(omega/(2*a))

#pas de discrétisation
dx = 
dt = 

#paramètres du maillage
Nx = int(L/dx)
Nt = int(t_fin/dt)

#coefficient de discrétisation
alpha = (D_Alg*dt)/(dx**2)

#conditions aux limites et initiales 
Ts = 

Implémentation du schéma Crank-Nicolson : 

In [ ]:

    #matrice du schéma de Crank-Nicolson (ATn+1 = BTn)
    A = triadiag(Nx,-alpha,2+2*alpha)
    B = triadiag(Nx,alpha,2-2*alpha)
    